Full pandas dataframe created for all shep race results

todo:
create graph of box number vs place
create lin model of distance box vs time/place?

idea:
clustering analysis of dog to predict dogs that will behave the same  

working:
currently working with all split time correct in r
creating dog index in r to hold previous race results


In [1]:
import os
import xml.etree.ElementTree as ET
import pandas as pd
import tqdm
import pickle

In [10]:
#Loading race results database
os.chdir(r'C:\Users\Nick\Documents\GitHub\grvmodel\grv scraper\full race results')
file_list = os.listdir()
path = os.getcwd()


file_list

['rr-BAL01Apr17.xml',
 'rr-BAL01Apr19.xml',
 'rr-BAL01Aug16.xml',
 'rr-BAL01Aug18.xml',
 'rr-BAL01Dec18.xml',
 'rr-BAL01Feb16.xml',
 'rr-BAL01Feb17.xml',
 'rr-BAL01Feb21.xml',
 'rr-BAL01Jan18.xml',
 'rr-BAL01Jan20.xml',
 'rr-BAL01Jul19.xml',
 'rr-BAL01Mar17.xml',
 'rr-BAL01Mar21.xml',
 'rr-BAL01May17.xml',
 'rr-BAL01Nov17.xml',
 'rr-BAL01Oct18.xml',
 'rr-BAL01Sep19.xml',
 'rr-BAL02Apr18.xml',
 'rr-BAL02Aug19.xml',
 'rr-BAL02Dec16.xml',
 'rr-BAL02Dec17.xml',
 'rr-BAL02Dec19.xml',
 'rr-BAL02Jan17.xml',
 'rr-BAL02Jan19.xml',
 'rr-BAL02Jul18.xml',
 'rr-BAL02Jun21.xml',
 'rr-BAL02May17.xml',
 'rr-BAL02Nov16.xml',
 'rr-BAL02Oct17.xml',
 'rr-BAL02Oct19.xml',
 'rr-BAL03Apr17.xml',
 'rr-BAL03Apr19.xml',
 'rr-BAL03Aug16.xml',
 'rr-BAL03Aug17.xml',
 'rr-BAL03Dec18.xml',
 'rr-BAL03Feb16.xml',
 'rr-BAL03Feb21.xml',
 'rr-BAL03Jan18.xml',
 'rr-BAL03Jul17.xml',
 'rr-BAL03Jul19.xml',
 'rr-BAL03Jun17.xml',
 'rr-BAL03Jun19.xml',
 'rr-BAL03Mar18.xml',
 'rr-BAL03Mar21.xml',
 'rr-BAL03May18.xml',
 'rr-BAL03

In [14]:
for i in tqdm.tqdm(file_list):
    root = ET.parse(i).getroot()

    track = root[0]
    #print("track =", track.text)
    date = root[1]
    #print("date =", date.text)


    for race in root.iter('Race'):
        #print("race id = ",race.attrib["id"])
        #print("Distance =", race[3].text, "\n")
        #print("Grade =", race[4].text, "\n")
        #print("place name box split split_margin runtime")
        places = []
        ids = []
        boxes = []
        weights = []
        margins = []
        splittimes = []
        split_margin = []
        split_position = []
        runtimes = []

        for dog in race.iter('Dog'):
            if not dog.attrib["id"]:
                continue
            if not dog[0].text:
                continue
            if (dog[0].text in "RSBFNPTBD")or('=' in dog[0].text):
                #print("scratched check working")
                continue
            else:
                places.append(dog[0].text)
                weights.append(dog[5].text)
                margins.append(dog[8].text)
                ids.append(dog.attrib["id"])
                boxes.append(dog[3].text)
                splittimes.append(dog[13].text)
                runtimes.append(dog[14].text)
                print(dog[8].text)
                break
            break
        break
    break

  0%|          | 0/4811 [00:00<?, ?it/s]

0.20


In [17]:
master_list = result_getter(file_list, path)
master_list

100%|██████████| 4811/4811 [00:33<00:00, 141.82it/s]


[('208756700',
  '01 Apr 17',
  'Ballarat',
  '450m',
  'Maiden Heat',
  '1',
  '1296100098',
  '2',
  '6.73',
  '0.00',
  '26.17',
  '30.2',
  '0.20'),
 ('208756700',
  '01 Apr 17',
  'Ballarat',
  '450m',
  'Maiden Heat',
  '2',
  '1177320005',
  '3',
  '6.87',
  '0.14',
  '26.18',
  '29.3',
  '0.20'),
 ('208756700',
  '01 Apr 17',
  'Ballarat',
  '450m',
  'Maiden Heat',
  '3',
  '1549980026',
  '5',
  '6.99',
  '0.26',
  '26.26',
  '29.4',
  '1.24'),
 ('208756700',
  '01 Apr 17',
  'Ballarat',
  '450m',
  'Maiden Heat',
  '4',
  '1474670055',
  '7',
  '6.88',
  '0.15',
  '26.36',
  '32.8',
  '2.79'),
 ('208756700',
  '01 Apr 17',
  'Ballarat',
  '450m',
  'Maiden Heat',
  '5',
  '913850094',
  '8',
  '6.91',
  '0.18',
  '26.42',
  '32.4',
  '3.60'),
 ('208756700',
  '01 Apr 17',
  'Ballarat',
  '450m',
  'Maiden Heat',
  '6',
  '1334020065',
  '6',
  '6.97',
  '0.24',
  '26.67',
  '32.6',
  '7.14'),
 ('208756700',
  '01 Apr 17',
  'Ballarat',
  '450m',
  'Maiden Heat',
  '7',
  '10

In [3]:
def result_labels(file_list, path):
    label_list = []
    for i in file_list:
        root = ET.parse(i).getroot()
        
        for race in root.iter('Race'):
            race_id = race.attrib["id"]
            print("race id = ",race_id)
            
            places = []
                        
            for dog in race.iter('Dog'):
                places.append(dog[0].text)
            print(places)
        break
            

In [15]:
def result_getter(file_list, path):
    master_list = []
    
    for i in tqdm.tqdm(file_list):
        root = ET.parse(i).getroot()

        track = root[0]
        #print("track =", track.text)
        date = root[1]
        #print("date =", date.text)


        for race in root.iter('Race'):
            #print("race id = ",race.attrib["id"])
            #print("Distance =", race[3].text, "\n")
            #print("Grade =", race[4].text, "\n")
            #print("place name box split split_margin runtime")
            places = []
            ids = []
            boxes = []
            splittimes = []
            split_margin = []
            weights = []
            margins = []
            split_position = []
            runtimes = []

            for dog in race.iter('Dog'):
                if not dog.attrib["id"]:
                    continue
                if not dog[0].text:
                    continue
                if (dog[0].text in "RSBFNPTBD")or('=' in dog[0].text):
                    #print("scratched check working")
                    continue
                else:
                    places.append(dog[0].text)
                    ids.append(dog.attrib["id"])
                    boxes.append(dog[3].text)
                    splittimes.append(dog[13].text)
                    runtimes.append(dog[14].text)
                    weights.append(dog[5].text)
                    margins.append(dog[8].text)

            for i in range(len(splittimes)):
                if splittimes[0] == None:
                    split_margin.append(pd.NaT)
                    continue
                try:
                    split_margin.append("%.2f" % (float(splittimes[i])-float(min(splittimes))))
                    
                                        #trying to get split position
                except:
                    split_margin.append(pd.NaT)
                    
                    
            for i in range(len(places)):
                try:
                    pass
                    #print(places[i], ids[i], boxes[i], splittimes[i], "%.2f" % split_margin[i], runtimes[i])
                except :
                    pass
                    #print(places[i], ids[i], boxes[i], splittimes[i], split_margin[i], runtimes[i])

            #print("\n------------------------\n")
            zipall = list(zip(places,ids,boxes,splittimes,split_margin,runtimes,weights,margins))
            #print(zipall)
            for i in range(len(zipall)):
                zipall[i] = (race.attrib["id"],date.text,track.text, race[3].text, race[4].text,) + zipall[i]
                master_list.append(zipall[i])

    return master_list

In [18]:
col_names = ["race_id", "date", "track","dist","grade", "place", "dog_id", "box", "split_times", "split_margins", "run_time", "weigth", "margin"]
df = pd.DataFrame(master_list, columns = col_names)
df.place = df.place.astype(float)
df.place.unique()

array([1., 2., 3., 4., 5., 6., 7., 8.])

In [15]:
print(df.size)
df = df.dropna()
print(df.size)
df.to_csv("masterfile.csv")

with open("results-df.npy", "wb") as fp:   #Pickling
    
    pickle.dump(df, fp)

4596317
4566276


In [19]:
df.grade.value_counts()

Grade 5                            92157
Mixed 4/5                          38487
Maiden                             32138
Grade 5 T3                         26966
Mixed 6/7                          22233
Grade 7                            17453
Restricted Win                     16629
Grade 5 Heat                       14403
Tier 3 - Maiden                    13181
Maiden Heat                        12606
Restricted Win Heat                11648
Tier 3 - Grade 7                   10824
Grade 6                            10476
S/E Heat                            8932
Free For All                        8421
Mixed 6/7 Heat                      8317
Tier 3 - Restricted Win             6760
Tier 3 - Grade 6                    6703
Grade 5 No Penalty                  6357
Aged Prize Money                    5449
Grade 5 Final                       4533
Special Event                       4440
Mixed 3/4                           3869
Restricted Win Final                3782
Maiden Final    

In [24]:
#generates adjusted place to transfer to form results

dfsorted = df.sort_values(['box'])
dfsorted['place_adj'] = 1/(dfsorted["place"])
grouped = dfsorted.groupby(['race_id'])

for group in grouped:
    
    print(group[1])
    break
 

         race_id       date      track  dist         grade  place      dog_id  \
349113  12539914  22 Jun 18  Traralgon  525m  Maiden Final    8.0   109032142   
349109  12539914  22 Jun 18  Traralgon  525m  Maiden Final    4.0   145856635   
349111  12539914  22 Jun 18  Traralgon  525m  Maiden Final    6.0  2131520028   
349108  12539914  22 Jun 18  Traralgon  525m  Maiden Final    3.0   167542442   
349107  12539914  22 Jun 18  Traralgon  525m  Maiden Final    2.0   128006011   
349112  12539914  22 Jun 18  Traralgon  525m  Maiden Final    7.0  1199430051   
349110  12539914  22 Jun 18  Traralgon  525m  Maiden Final    5.0   159734832   
349106  12539914  22 Jun 18  Traralgon  525m  Maiden Final    1.0   115367782   

       box split_times split_margins run_time weigth margin  place_adj  
349113   1        5.06          0.20    32.31   26.8  17.64   0.125000  
349109   2        5.06          0.20    31.53   31.2   6.51   0.250000  
349111   3        5.06          0.20    31.70   23.

In [25]:
dfsorted.to_csv("masterfilenew.csv")


In [23]:
df.head(7).sort_values(['place'])

,race_id,date,track,dist,grade,place,dog_id,box,split_times,split_margins,run_time,weigth,margin
0,208756700,01 Apr 17,Ballarat,450m,Maiden Heat,1.0,1296100098,2,6.73,0.00,26.17,30.2,0.20
1,208756700,01 Apr 17,Ballarat,450m,Maiden Heat,2.0,1177320005,3,6.87,0.14,26.18,29.3,0.20
2,208756700,01 Apr 17,Ballarat,450m,Maiden Heat,3.0,1549980026,5,6.99,0.26,26.26,29.4,1.24
3,208756700,01 Apr 17,Ballarat,450m,Maiden Heat,4.0,1474670055,7,6.88,0.15,26.36,32.8,2.79
4,208756700,01 Apr 17,Ballarat,450m,Maiden Heat,5.0,913850094,8,6.91,0.18,26.42,32.4,3.60
5,208756700,01 Apr 17,Ballarat,450m,Maiden Heat,6.0,1334020065,6,6.97,0.24,26.67,32.6,7.14
6,208756700,01 Apr 17,Ballarat,450m,Maiden Heat,7.0,1096450034,1,6.92,0.19,26.87,29.9,10.03
